# Weights & Biases

Linear regression with better tracking

In [1]:
%matplotlib inline
import random
import torch
import torch.nn as nn
import wandb

Let's define the hyper parameters used for training so we can log them and keep track of it

In [2]:
lr = 3e-3
epochs = 2000
wandb.init(project="example", config={"lr": lr, "epochs": epochs})

wandb: Currently logged in as: ingambe (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


As before, let's generate some synthetic data and train on it!

In [3]:
true_W = torch.Tensor([2.5, 4.8])
input_data = torch.randn((10, 2))
ground_truth = torch.matmul(input_data, true_W)
ground_truth, true_W

(tensor([-15.5195,   4.9580,  13.6744,   1.0959,   1.6048,   7.6192, -12.1037,
          -2.5858,   4.8203,  -3.3834]),
 tensor([2.5000, 4.8000]))

In [4]:
random_weights = torch.randn((2), requires_grad=True)
criterion = nn.MSELoss()
for i in range(epochs):
    prediction = torch.matmul(input_data, random_weights)
    loss = criterion(prediction, ground_truth)
    loss.backward()
    # Why do we need to remove the gradient computation here ?
    with torch.no_grad():
        random_weights = random_weights - lr * random_weights.grad
        wandb.log({'loss': loss.item()})
    random_weights.requires_grad = True

We can even plot the histogram of the weights, this is usefull to know the importance of each input features

In [5]:
labels = ["weight_1", "weight_2"]
data = [[label, val] for (label, val) in zip(labels, random_weights.detach().numpy())]
table = wandb.Table(data=data, columns = ["label", "value"])
wandb.log({"learned_weights" : wandb.plot.bar(table, "label",
                               "value", title="Output weights")})